In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Gooogle-Stock-Price-V5'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
data = data['High'].tolist()

In [4]:
data = torch.from_numpy(np.array(data)).to(device).view(1,-1).float()

In [5]:
data

tensor([[332.8300, 333.8700, 330.7500,  ..., 794.2300, 785.9300, 782.7800]],
       device='cuda:0')

In [6]:
data_input = data[:,:-1]
data_target = data[:,1:]

In [7]:
data_input

tensor([[332.8300, 333.8700, 330.7500,  ..., 797.8600, 794.2300, 785.9300]],
       device='cuda:0')

In [8]:
data_target

tensor([[333.8700, 330.7500, 328.7700,  ..., 794.2300, 785.9300, 782.7800]],
       device='cuda:0')

In [9]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.hidden = 512
        self.lstm1 = LSTMCell(1,self.hidden)
        self.lstm2 = LSTMCell(self.hidden,self.hidden)
        self.linear = Linear(self.hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.size(0)
        h_t1,c_t1 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        h_t2,c_t2 = torch.zeros(batch_size,self.hidden).to(device),torch.zeros(batch_size,self.hidden).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,h_t2))
            pred = self.linear(h_t2)
            preds.append(pred)
        for _ in range(future):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,h_t2))
            pred = self.linear(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [10]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [11]:
data_input.shape

torch.Size([1, 1257])

In [18]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        loss.backward()
        wandb.log({'Loss':loss.item()})
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        n = data_input.shape[1]
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].view(-1).cpu().detach().numpy()
        plt.figure(figsize=(12,6))
        plt.plot(preds[n:])
        plt.plot(data_target.view(-1).cpu().detach().numpy())
        plt.savefig('./img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('./img.png'))})
wandb.finish()

Loss,312821.25
_runtime,7
_timestamp,1634222899
_step,0


Loss,▁
_runtime,▁
_timestamp,▁
_step,▁


wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


100%|█████████████████████████████████████████| 100/100 [01:27<00:00,  1.14it/s]


Loss,251811.59375
_runtime,94
_timestamp,1634223078
_step,299
Val Loss,251343.375


Loss,██▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Val Loss,██▇▇▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁


In [19]:
torch.save(data,'./data.pt')
torch.save(data,'./data.pth')
torch.save(data_input,'data_input.pt')
torch.save(data_input,'data_input.pth')
torch.save(data_target,'data_target.pt')
torch.save(data_target,'data_target.pth')
torch.save(model,'custom-model.pt')
torch.save(model,'custom-model.pth')
torch.save(model.state_dict(),'custom-model-sd.pt')
torch.save(model.state_dict(),'custom-model-sd.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')
torch.save(model,'custom-model.pt')
torch.save(model,'custom-model.pth')
torch.save(model.state_dict(),'custom-model-sd.pt')
torch.save(model.state_dict(),'custom-model-sd.pth')
torch.save(model,'model.pt')
torch.save(model,'model.pth')
torch.save(model.state_dict(),'model-sd.pt')
torch.save(model.state_dict(),'model-sd.pth')